# Titanic Survival Prediction

> ❗ Este modelo é fortemente inspirado [neste post da documentação do Danfo.](https://danfo.jsdata.org/examples/titanic-survival-prediction-using-danfo.js-and-tensorflow.js)
> 

### Importar *libs*

In [9]:
import dfd from 'danfojs-node';

const tf = dfd.tensorflow;

TypeError: Unable to get CWD
  Caused by:
    No such file or directory (os error 2)

### Carregar dados

Os dados serão carregados de um `csv`, pré-baixado no diretório `./data` e que foi [obtido da Universidade de Stanford](https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv).

In [5]:


const load_data = async () => await dfd.readCSV( "./data/titanic.csv" );

const df = await load_data();

df.head().print(); // printa as primeiras 5 linhas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Name              │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ Mr. Owen Harris…  │ male              │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ Mrs. John Bradl…  │ female            │ 38                │ 1         

### Investigar os dados

Vamos operar alguns comandos para entender melhor os dados que temos em mãos.

In [6]:
df.ctypes.print(); // printa os tipos de dados de cada coluna

╔═════════════════════════╤═════════╗
║ Survived                │ int32   ║
╟─────────────────────────┼─────────╢
║ Pclass                  │ int32   ║
╟─────────────────────────┼─────────╢
║ Name                    │ string  ║
╟─────────────────────────┼─────────╢
║ Sex                     │ string  ║
╟─────────────────────────┼─────────╢
║ Age                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Siblings/Spouses Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Parents/Children Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Fare                    │ float32 ║
╚═════════════════════════╧═════════╝



Vemos que temos duas colunas que são do tipo `string` ('Name' e 'Sex'). 

Para nosso modelo, precisamos que todas as colunas sejam numéricas. As redes neurais que vamos utilizar não conseguem lidar com strings muito bem.

Mas antes de resolver esse problema, vamos ter um panorama estatístico dos dados.

In [11]:
df.describe().print(); // printa estatísticas descritivas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 887               │ 887               │ 887               │ 887               │ 887               │ 887               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 0.3855693348365…  │ 2.3055242390078…  │ 29.471443066516…  │ 0.5253664036076…  │ 0.3833145434047…  │ 32.305420180383…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ std        │ 0.4870041177510…  │ 0.8366620036697…  │ 